# Вариант 7. Гибкость юношей первого курса в зависимости от окружности грудной клетки в покое, роста, времени 10 хлопков.

Данные из файла Volgmed_2013.xls 
(конкретные данные выбрать в соответствии с вариантом) 
очистить от выбросов и пропусков, найти коэффициенты линейной регрессии 
с их пи-значениями и коэффициент детерминации. 

Исключить регрессоры, у которых пи-значения больше 0,05, 
вычислить заново коэффициенты линейной регрессии с их пи-значениями и 
коэффициент детерминации, а также 
выборочную дисперсию регрессионных остатков. 

Провести кросс-валидацию, разбив 200 раз случайным образом выборку 
на обучающую (80%) и контрольную (20%) группы.
Посчитать среднее значение квадрата отклонения по контрольной группе и 
его отношение к выборочной дисперсии регрессионных остатков.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

In [2]:
FILENAME = "Volgmed_2013.xlsx"

datapath = Path(FILENAME)
df = pd.read_excel(datapath, index_col=0, header=1)

df.head()

,Пол,Год рождения,Факультет,Курс,Группа,Год поступления,Дата обследования,"Рост, см","Вес, кг","Окружность грудной клетки в покое, см",...,"Гибкость, см","Время 10 хлопков, с","Прыжок в длину с места, см","Челночный бег, с","Силовой норматив, ед","Бег 100 м, с","Бег 2-3 км, мин","Статическая выносливость, с","Задержка дыхания, с","Равновесие, с"
№ п/п,,,,,,,,,,,,,,,,,,,,,
104.0,жен,1981.0,лечебный,1.0,NaN,1999,2000-05-01,176,74.5,90,...,22,6.6,155,26,38,17.7,8.7,140,59.0,90.0
104.0,жен,1981.0,лечебный,3.0,NaN,1999,2002-05-01,176,74,90,...,23,6.5,158,25,47,18.7,6.58,144,68.0,90.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['Пол', 'Год рождения', 'Факультет', 'Курс', 'Группа', 'Год поступления',
       'Дата обследования', 'Рост, см', 'Вес, кг',
       'Окружность грудной клетки в покое, см',
       'Окружность грудной клетки на вдохе, см',
       'Окружность грудной клетки на выдохе, см',
       'Экскурсия грудной клетки, см', 'Толщина кожной складки, мм',
       'Сила правой руки, кг', 'Сила левой руки, кг', 'Становая сила, кг',
       'Гибкость, см', 'Время 10 хлопков, с', 'Прыжок в длину с места, см',
       'Челночный бег, с', 'Силовой норматив, ед', 'Бег 100 м, с',
       'Бег 2-3 км, мин', 'Статическая  выносливость, с',
       'Задержка дыхания, с', 'Равновесие, с'],
      dtype='object')

In [4]:
COLUMNS = [
    "Окружность грудной клетки в покое, см",
    "Рост, см",
    "Время 10 хлопков, с",
    "Гибкость, см",
]

In [5]:
data = df[(df["Курс"] == 1) & ((df["Пол"] == "муж") | (df["Пол"] == "муж."))][COLUMNS]
data.reset_index(drop=True, inplace=True)
# data.rename(columns={"Гибкость, см": "flexibility", "Вес, кг": "weight"}, inplace=True)

data

,"Окружность грудной клетки в покое, см","Рост, см","Время 10 хлопков, с","Гибкость, см"
0,84,174,5.2,11
1,85,179,5.4,4
2,119,182,5.5,17
3,80,170,6.8,13
4,77,171,6,7
...,...,...,...,...
276,87,177,5.8,18
277,86,187,5.8,15
278,86,162.5,6,18
279,90,181.5,5.9,10


In [6]:
data.dtypes

Окружность грудной клетки в покое, см    object
Рост, см                                 object
Время 10 хлопков, с                      object
Гибкость, см                             object
dtype: object

In [7]:
data = data.apply(pd.to_numeric, errors='coerce')

data.dtypes

Окружность грудной клетки в покое, см    float64
Рост, см                                 float64
Время 10 хлопков, с                      float64
Гибкость, см                             float64
dtype: object

In [8]:
q1 = data.quantile(0.25)
q3 = data.quantile(0.75)
iqr = q3 - q1
outliers = (data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr))
data = data[~outliers.any(axis=1)]

data.dropna(inplace=True)

data

,"Окружность грудной клетки в покое, см","Рост, см","Время 10 хлопков, с","Гибкость, см"
0,84.0,174.0,5.2,11.0
1,85.0,179.0,5.4,4.0
3,80.0,170.0,6.8,13.0
4,77.0,171.0,6.0,7.0
5,82.0,173.0,6.1,18.0
...,...,...,...,...
276,87.0,177.0,5.8,18.0
277,86.0,187.0,5.8,15.0
278,86.0,162.5,6.0,18.0
279,90.0,181.5,5.9,10.0


In [9]:
x = sm.add_constant(data[["Окружность грудной клетки в покое, см", "Рост, см", "Время 10 хлопков, с",]])
y = data["Гибкость, см"]
model = sm.OLS(y, x).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Гибкость, см   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.893
Date:                Wed, 30 Oct 2024   Prob (F-statistic):              0.131
Time:                        23:19:40   Log-Likelihood:                -836.15
No. Observations:                 258   AIC:                             1680.
Df Residuals:                     254   BIC:                             1695.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [10]:
x_filtered = data["Окружность грудной клетки в покое, см"]
model = sm.OLS(y, x_filtered).fit()

print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:           Гибкость, см   R-squared (uncentered):                   0.791
Model:                            OLS   Adj. R-squared (uncentered):              0.790
Method:                 Least Squares   F-statistic:                              973.9
Date:                Wed, 30 Oct 2024   Prob (F-statistic):                    2.14e-89
Time:                        23:19:40   Log-Likelihood:                         -836.54
No. Observations:                 258   AIC:                                      1675.
Df Residuals:                     257   BIC:                                      1679.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------

In [15]:
# Провести кросс-валидацию, разбив 200 раз случайным образом выборку на обучающую (80%) и контрольную (20%) группы.
# Посчитать среднее значение квадрата отклонения по контрольной группе и его отношение к выборочной дисперсии регрессионных остатков.

n = 200
squared_deviation = []
for _ in range(n):
    data_train = data.sample(frac=0.8)
    data_test = data.drop(data_train.index)

    x_train = sm.add_constant(data_train["Окружность грудной клетки в покое, см"])
    y_train = data_train["Гибкость, см"]
    x_test = sm.add_constant(data_test["Окружность грудной клетки в покое, см"])
    y_test = data_test["Гибкость, см"]

    model = sm.OLS(y_train, x_train).fit()
    y_pred = model.predict(x_test)
    squared_deviation.append(np.mean((y_test - y_pred) ** 2))

squared_deviation = np.array(squared_deviation)
variance = np.var(model.resid)
mean_squared_deviation = np.mean(squared_deviation)
ratio = mean_squared_deviation / variance

print(f"Среднее значение квадрата отклонения (MSE) по контрольной группе: {mean_squared_deviation}")
print(f"Выборочная дисперсия регрессионных остатков: {variance}")
print(f"Отношение MSE к выборочной дисперсии: {ratio}")

Среднее значение квадрата отклонения (MSE) по контрольной группе: 38.41054734624333
Выборочная дисперсия регрессионных остатков: 36.58062187883014
Отношение MSE к выборочной дисперсии: 1.0500244493785438
